필요한 라이브러리 import

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from tqdm import tqdm
from nltk import sent_tokenize

감정단어 사전 load

In [2]:
# 데이터 출처: https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm
with open('./data/Korean-NRC-EmoLex.txt', 'r', encoding = 'utf-8') as f:
    emolex = f.readlines()

In [3]:
column_name = emolex[0].strip().split('\t')
# 10개의 속성 중 하나 이상 해당되는 단어만 수집
emolex_split = [x.strip().split('\t') for x in emolex[1:] if sum(map(int, x.strip().split('\t')[1:-1])) > 0]

NRC_EmoLex = pd.DataFrame(emolex_split, columns = column_name)

In [4]:
for c in column_name[1:-1]:
    NRC_EmoLex[c] = NRC_EmoLex[c].astype(int)

NRC_EmoLex = NRC_EmoLex[['Korean Word', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']]

In [5]:
NRC_EmoLex

,Korean Word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,주판,0,0,0,0,0,0,0,0,0,1
1,버리다,0,0,0,1,0,1,0,1,0,0
2,버려진,1,0,0,1,0,1,0,1,0,0
3,포기,1,0,0,1,0,1,0,1,1,0
4,아바,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6448,익살스러운,0,0,0,0,0,0,0,0,1,0
6449,열성,0,1,0,0,1,0,1,0,1,1
6450,열심,0,0,0,0,1,0,1,0,0,1
6451,풍미,0,1,0,0,1,0,1,0,0,1


In [6]:
kor = list(NRC_EmoLex['Korean Word']) # 한국어 단어만 추출
kor[:10]

['주판', '버리다', '버려진', '포기', '아바', '대 수도 원장', '유 괴', '이상한', '수차', '혐오하다']

에세이 번역 데이터 load

In [7]:
data = pd.read_csv('./data/essays_kor.csv')[['TEXT_KOR', 'cEXT', 'cNEU', 'cCON']]
data

,TEXT_KOR,cEXT,cNEU,cCON
0,"음, 지금은 낮잠에서 막 일어났어요. 좀 이상하긴 한데, 텍사스로 이사 온 이후로 ...",n,y,n
1,"자, 이제 의식의 흐름 에세이를 시작하겠습니다. 고등학교 때 가끔 이런 걸 했었어요...",n,n,n
2,"누를 수 있는 열린 키보드와 버튼. 마침내 효과가 있었고 나는 마침표, 쉼표 및 모...",n,y,y
3,믿을 수가 없는 일이야! 정말 일어나고 있어요! 맥박이 미친 듯이 뛰고 있어요....,y,n,y
4,"자, 여기 다시 한번 좋은 오래된 의식 할당에 대해 말씀드리겠습니다. 다시 HS 영...",y,n,n
...,...,...,...,...
2462,집에 왔어요. 자고 싶었는데 다음 주 중으로 끝내야 할 심리학 숙제가 생각났어요. ...,n,y,y
2463,의식의 흐름 skdj. 그것의 철자가 어떻게 되나요? 내가 알기나 하면. 나는 오늘...,y,y,n
2464,오늘은 12월 8일 수요일이고 이번 학기에는 많은 일들이 있었다. 한 학기를 힘차게...,n,n,n
2465,이번 주는 지옥 같았어 아무튼 이제 20분짜리 글쓰기 과제 시간이에요. 나는 지금 ...,n,y,n


In [8]:
token = data['TEXT_KOR'].apply(sent_tokenize)
token

0       [음, 지금은 낮잠에서 막 일어났어요., 좀 이상하긴 한데, 텍사스로 이사 온 이후...
1       [자, 이제 의식의 흐름 에세이를 시작하겠습니다., 고등학교 때 가끔 이런 걸 했었...
2       [누를 수 있는 열린 키보드와 버튼., 마침내 효과가 있었고 나는 마침표, 쉼표 및...
3       [믿을 수가 없는 일이야!, 정말 일어나고 있어요!, 맥박이 미친 듯이 뛰고 있어요...
4       [자, 여기 다시 한번 좋은 오래된 의식 할당에 대해 말씀드리겠습니다., 다시 HS...
                              ...                        
2462    [집에 왔어요., 자고 싶었는데 다음 주 중으로 끝내야 할 심리학 숙제가 생각났어요...
2463    [의식의 흐름 skdj., 그것의 철자가 어떻게 되나요?, 내가 알기나 하면., 나...
2464    [오늘은 12월 8일 수요일이고 이번 학기에는 많은 일들이 있었다., 한 학기를 힘...
2465    [이번 주는 지옥 같았어 아무튼 이제 20분짜리 글쓰기 과제 시간이에요., 나는 지...
2466    [방금 브래디와 통화를 끝냈어요., 나는 우리가 이번 주말에 정확히 무엇을 할지 결...
Name: TEXT_KOR, Length: 2467, dtype: object

문장별 분리<br>
https://wglee87.github.io/2020/07/13/Pandas-%ED%95%9C-%EC%85%80%EC%9D%98-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC-%EC%97%AC%EB%9F%AC-%ED%96%89%EC%9C%BC%EB%A1%9C-%EB%82%98%EB%88%84%EA%B8%B0/

In [9]:
df = token.apply(lambda x: pd.Series(x))
df = df.stack().reset_index(level=1, drop=True).to_frame('TEXT_SPL')
df

,TEXT_SPL
0,"음, 지금은 낮잠에서 막 일어났어요."
0,"좀 이상하긴 한데, 텍사스로 이사 온 이후로 일에 집중하는 데 문제가 생겼어요."
0,나는 10학년 때 4시를 치자마자 숙제를 시작해서 끝날 때까지 멈추지 않았던 것을 ...
0,물론 더 쉬웠지만 그래도 해냈어요.
0,"하지만 제가 여기로 이사 왔을 때, 숙제는 조금 더 어려워졌고, 바쁜 일이 훨씬 더..."
...,...
2466,"응, 크리스틴이 돌아왔고 우리는 지금 먹으러 갈 수 있어."
2466,내 시간은 거의 끝났어.
2466,나도 바지에 오줌 쌀 것 같아.
2466,AHHHHH!!!!!


In [10]:
result = data.merge(df, left_index=True, right_index=True, how='left')
result.reset_index(inplace = True, drop=True)
result

,TEXT_KOR,cEXT,cNEU,cCON,TEXT_SPL
0,"음, 지금은 낮잠에서 막 일어났어요. 좀 이상하긴 한데, 텍사스로 이사 온 이후로 ...",n,y,n,"음, 지금은 낮잠에서 막 일어났어요."
1,"음, 지금은 낮잠에서 막 일어났어요. 좀 이상하긴 한데, 텍사스로 이사 온 이후로 ...",n,y,n,"좀 이상하긴 한데, 텍사스로 이사 온 이후로 일에 집중하는 데 문제가 생겼어요."
2,"음, 지금은 낮잠에서 막 일어났어요. 좀 이상하긴 한데, 텍사스로 이사 온 이후로 ...",n,y,n,나는 10학년 때 4시를 치자마자 숙제를 시작해서 끝날 때까지 멈추지 않았던 것을 ...
3,"음, 지금은 낮잠에서 막 일어났어요. 좀 이상하긴 한데, 텍사스로 이사 온 이후로 ...",n,y,n,물론 더 쉬웠지만 그래도 해냈어요.
4,"음, 지금은 낮잠에서 막 일어났어요. 좀 이상하긴 한데, 텍사스로 이사 온 이후로 ...",n,y,n,"하지만 제가 여기로 이사 왔을 때, 숙제는 조금 더 어려워졌고, 바쁜 일이 훨씬 더..."
...,...,...,...,...,...
127387,방금 브래디와 통화를 끝냈어요. 나는 우리가 이번 주말에 정확히 무엇을 할지 결정하...,n,y,n,"응, 크리스틴이 돌아왔고 우리는 지금 먹으러 갈 수 있어."
127388,방금 브래디와 통화를 끝냈어요. 나는 우리가 이번 주말에 정확히 무엇을 할지 결정하...,n,y,n,내 시간은 거의 끝났어.
127389,방금 브래디와 통화를 끝냈어요. 나는 우리가 이번 주말에 정확히 무엇을 할지 결정하...,n,y,n,나도 바지에 오줌 쌀 것 같아.
127390,방금 브래디와 통화를 끝냈어요. 나는 우리가 이번 주말에 정확히 무엇을 할지 결정하...,n,y,n,AHHHHH!!!!!


감정단어 미포함 문장 인덱스 추출

In [11]:
del_list = []
spl = result.TEXT_SPL

for i in tqdm(range(len(spl))):
    check = False
    target = spl[i]
    for k in kor:
        if k in target:
            check = True
            break
    if check == False:
        del_list.append(i)

100%|████████████████████████████████████████████████████████████████████████| 127392/127392 [00:21<00:00, 5917.21it/s]


In [12]:
print(len(del_list))
print(del_list)

9190
[26, 27, 28, 49, 64, 65, 67, 70, 77, 89, 95, 111, 113, 114, 118, 130, 134, 145, 150, 157, 158, 170, 184, 186, 196, 198, 200, 202, 203, 205, 208, 209, 211, 218, 235, 259, 274, 279, 289, 290, 293, 302, 306, 315, 324, 328, 334, 336, 337, 345, 347, 348, 349, 357, 360, 369, 373, 377, 378, 384, 410, 417, 420, 454, 458, 500, 519, 530, 591, 595, 608, 615, 621, 641, 648, 652, 653, 723, 725, 748, 753, 754, 755, 757, 763, 764, 771, 776, 780, 789, 791, 800, 804, 805, 814, 822, 826, 828, 846, 918, 928, 943, 983, 987, 988, 995, 1021, 1066, 1076, 1089, 1119, 1128, 1141, 1143, 1148, 1160, 1161, 1163, 1168, 1176, 1190, 1226, 1227, 1229, 1236, 1247, 1252, 1291, 1295, 1325, 1344, 1346, 1386, 1410, 1432, 1437, 1448, 1458, 1460, 1471, 1494, 1522, 1526, 1529, 1559, 1569, 1594, 1608, 1624, 1632, 1633, 1636, 1638, 1653, 1661, 1663, 1670, 1672, 1682, 1685, 1687, 1690, 1695, 1707, 1726, 1738, 1816, 1915, 1932, 1933, 1942, 1962, 1963, 1988, 1991, 2000, 2024, 2032, 2039, 2041, 2042, 2048, 2051, 2056, 2059, 2

감정단어 미포함 문장 제거

In [13]:
result.drop(del_list, axis=0, inplace=True)
result.reset_index(drop=True, inplace=True)
result = result[['TEXT_SPL', 'cEXT', 'cNEU', 'cCON']]
result.rename(columns = {'TEXT_SPL':'TEXT_KOR'}, inplace=True)
result

,TEXT_KOR,cEXT,cNEU,cCON
0,"음, 지금은 낮잠에서 막 일어났어요.",n,y,n
1,"좀 이상하긴 한데, 텍사스로 이사 온 이후로 일에 집중하는 데 문제가 생겼어요.",n,y,n
2,나는 10학년 때 4시를 치자마자 숙제를 시작해서 끝날 때까지 멈추지 않았던 것을 ...,n,y,n
3,물론 더 쉬웠지만 그래도 해냈어요.,n,y,n
4,"하지만 제가 여기로 이사 왔을 때, 숙제는 조금 더 어려워졌고, 바쁜 일이 훨씬 더...",n,y,n
...,...,...,...,...
118197,"여기서 끝나는 게 아니라는 건 알지만, 내 인생에서 할 수 있는 일에는 선택의 여지...",n,y,n
118198,"응, 크리스틴이 돌아왔고 우리는 지금 먹으러 갈 수 있어.",n,y,n
118199,내 시간은 거의 끝났어.,n,y,n
118200,나도 바지에 오줌 쌀 것 같아.,n,y,n


성격별 데이터 분리 및 save

In [14]:
result[['TEXT_KOR', 'cEXT']].to_csv('E_essays.csv', encoding='utf-8-sig', index=None)
result[['TEXT_KOR', 'cNEU']].to_csv('N_essays.csv', encoding='utf-8-sig', index=None)
result[['TEXT_KOR', 'cCON']].to_csv('C_essays.csv', encoding='utf-8-sig', index=None)